## Importing our data
We use the ImageDataGenerator of Tensorflow to make the training, validation and testing sets

In [1]:
import os
import pandas
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen=ImageDataGenerator( rescale=1./255,rotation_range=0.2, shear_range=0.2,
                                  zoom_range=0.2, width_shift_range=0.2,
                                  height_shift_range=0.2, validation_split=0.2)
train_data=train_datagen.flow_from_directory('./prepared_data/train_data',target_size=(300,300),batch_size=80,class_mode='categorical',subset='training')
val_data=train_datagen.flow_from_directory('./prepared_data/train_data',target_size=(300,300),batch_size=80,class_mode='categorical',subset='validation')

test_datagen=ImageDataGenerator( rescale=1./255,rotation_range=0.2, shear_range=0.2,
                                  zoom_range=0.2, width_shift_range=0.2,
                                  height_shift_range=0.2)

test_data=test_datagen.flow_from_directory('./prepared_data/test_data',target_size=(300,300),batch_size=80,class_mode='categorical')

Found 11728 images belonging to 5 classes.
Found 2929 images belonging to 5 classes.
Found 3665 images belonging to 5 classes.


In [3]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM

## Making our Model
We make use of the vgg16 model in making our model. Because the imagenet weights were not trained on images related to our use case, we train it from scratch

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input

In [ ]:
model=Sequential()
model.add(Input(shape = (300,300,3)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
checkpoint = ModelCheckpoint('./model checkpoint/model.keras', monitor = 'val_accuracy', save_best_only = True)
earlystop = EarlyStopping(monitor = 'val_accuracy', patience = 7, restore_best_weights = True)
learning_rate = ReduceLROnPlateau(monitor = 'val_accuracy', patience = 3)
callbacks = [checkpoint, earlystop, learning_rate]

In [9]:
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(train_data, validation_data = val_data, 
          epochs = 15, 
          callbacks = callbacks)

Epoch 1/15


KeyboardInterrupt: 

In [ ]:
loss_train, acc_train = model.evaluate(train_data)
print(acc_train)
print(loss_train)

In [ ]:
loss_val, acc_val = model.evaluate(val_data)
print(acc_val)
print(loss_val)

In [ ]:
loss_test, acc_test = model.evaluate(test_data)
print(acc_test)
print(loss_test)

In [ ]:
y_pred = model.predict(test_data)
y_pred

In [10]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13754092627925743046
xla_global_id: -1
]
